# Unit 4: Create MoR tables

In unit 2, we learned about the Hudi table types. <br>In unit 3, we worked with the CoW table type called taxi_db.nyc_taxi_trips_hudi_cow.<br>
In this unit, we will create a MoR table off of the CoW table from module 3.

This module takes about 30 minutes to complete.

### Initialize Spark Session

In [1]:
spark = SparkSession.builder \
  .appName("Hudi-Learning-Unit-04-PySpark") \
  .master("yarn")\
  .enableHiveSupport()\
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog") \
  .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension") \
  .getOrCreate()

23/07/24 20:39:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark

### Declare & define variables

In [3]:
PROJECT_ID_OUTPUT=!gcloud config get-value core/project
PROJECT_ID=PROJECT_ID_OUTPUT[0]

In [4]:
PROJECT_NBR_OUTPUT=!gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
PROJECT_NBR=PROJECT_NBR_OUTPUT[0]

In [5]:
print(f"Project ID is {PROJECT_ID}")
print(f"Project Number is {PROJECT_NBR}")

Project ID is apache-hudi-lab
Project Number is 623600433888


In [6]:
PERSIST_TO_BUCKET = f"gs://gaia_data_bucket-{PROJECT_NBR}"
HUDI_COW_BASE_GCS_URI = f"{PERSIST_TO_BUCKET}/nyc-taxi-trips-hudi-cow"
HUDI_MOR_BASE_GCS_URI = f"{PERSIST_TO_BUCKET}/nyc-taxi-trips-hudi-mor"
DATABASE_NAME = "taxi_db"
COW_TABLE_NAME = "nyc_taxi_trips_hudi_cow"
MOR_TABLE_NAME = "nyc_taxi_trips_hudi_mor"

## 1. Create a MoR table for the lab

### 1.1. Review the tables in the managed Hive Metastore under taxi_db database

In [7]:
spark.sql("SHOW databases;").show()

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


+---------+
|namespace|
+---------+
|  default|
|  taxi_db|
+---------+



In [8]:
spark.sql("SHOW tables IN taxi_db;").show(truncate=False)

+---------+-----------------------+-----------+
|namespace|tableName              |isTemporary|
+---------+-----------------------+-----------+
|taxi_db  |nyc_taxi_trips_hudi_cow|false      |
+---------+-----------------------+-----------+



### 1.2. Create MoR table

In [9]:
 hudi_options = {
            'hoodie.database.name': DATABASE_NAME,
            'hoodie.table.name': MOR_TABLE_NAME,
            'hoodie.datasource.write.table.name': MOR_TABLE_NAME,
            'hoodie.datasource.write.table.type': 'MERGE_ON_READ',
            'hoodie.datasource.write.keygenerator.class':'org.apache.hudi.keygen.CustomKeyGenerator',
            'hoodie.datasource.write.recordkey.field': 'taxi_type,trip_year,trip_month,trip_day,vendor_id,pickup_datetime,dropoff_datetime,pickup_location_id,dropoff_location_id',
            'hoodie.datasource.write.partitionpath.field': 'trip_year:SIMPLE,trip_month:SIMPLE,trip_day:SIMPLE',
            'hoodie.datasource.write.precombine.field': 'pickup_datetime',
            'hoodie.datasource.write.hive_style_partitioning': 'true',
            'hoodie.partition.metafile.use.base.format': 'true', 
            'hoodie.datasource.write.drop.partition.columns': 'true'
        }

In [ ]:
# Reduce number of years in this list below to complete faster - leave at least two years
taxi_trip_years_list = [2019,2020,2021,2022]

In [10]:
for taxi_trip_year in taxi_trip_years_list:
    print(f"TRIP YEAR={taxi_trip_year}")
    tripsDF=spark.sql(f"SELECT * FROM taxi_db.nyc_taxi_trips_hudi_cow WHERE trip_year={taxi_trip_year}")
    tripsDF.write.format("hudi"). \
                options(**hudi_options). \
                mode("append"). \
                save(HUDI_MOR_BASE_GCS_URI)
    print("==============================")

TRIP YEAR=2019


23/07/24 20:40:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/07/24 20:40:31 WARN HoodieBackedTableMetadata: Metadata table was not found at path gs://gaia_data_bucket-623600433888/nyc-taxi-trips-hudi-mor/.hoodie/metadata


TRIP YEAR=2020


TRIP YEAR=2021


TRIP YEAR=2022


### 1.3. Run a count

In [12]:
tripsMorDF=spark.read.format("hudi").load(HUDI_MOR_BASE_GCS_URI)

In [13]:
tripsMorDF.count()

185550246

### 1.4. Review the Hudi metadata

In [14]:
! gsutil cat $HUDI_MOR_BASE_GCS_URI/.hoodie/hoodie.properties

#Updated at 2023-07-24T20:40:42.185997Z
#Mon Jul 24 20:40:42 UTC 2023
hoodie.compaction.payload.class=org.apache.hudi.common.model.OverwriteWithLatestAvroPayload
hoodie.table.type=MERGE_ON_READ
hoodie.table.metadata.partitions=files
hoodie.table.precombine.field=pickup_datetime
hoodie.table.partition.fields=trip_year,trip_month,trip_day
hoodie.archivelog.folder=archived
hoodie.timeline.layout.version=1
hoodie.table.checksum=4161037533
hoodie.datasource.write.drop.partition.columns=true
hoodie.table.timeline.timezone=LOCAL
hoodie.table.recordkey.fields=taxi_type,trip_year,trip_month,trip_day,vendor_id,pickup_datetime,dropoff_datetime,pickup_location_id,dropoff_location_id
hoodie.table.name=nyc_taxi_trips_hudi_mor
hoodie.partition.metafile.use.base.format=true
hoodie.datasource.write.hive_style_partitioning=true
hoodie.populate.meta.fields=true
hoodie.table.keygenerator.class=org.apache.hudi.keygen.CustomKeyGenerator
hoodie.table.base.file.format=PARQUET
hoodie.database.name=taxi_db
hood

### 1.5. Create an external table defintion in the Apache Hive Metastore

In [15]:
spark.sql(f"CREATE TABLE IF NOT EXISTS {DATABASE_NAME}.{MOR_TABLE_NAME} USING hudi LOCATION \"{HUDI_MOR_BASE_GCS_URI}\";").show()

23/07/24 21:26:48 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


++
||
++
++



In [16]:
spark.sql("SHOW tables IN taxi_db;").show(truncate=False)

+---------+-----------------------+-----------+
|namespace|tableName              |isTemporary|
+---------+-----------------------+-----------+
|taxi_db  |nyc_taxi_trips_hudi_cow|false      |
|taxi_db  |nyc_taxi_trips_hudi_mor|false      |
+---------+-----------------------+-----------+



23/07/24 21:26:49 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


### 1.6. Explore the MoR table

These are just some sample and very simple queries.  

In [17]:
spark.sql(f"SELECT trip_year,trip_month,trip_day, AVG(tip_amount) as avg_tips FROM {DATABASE_NAME}.{MOR_TABLE_NAME} WHERE trip_year=2021 and trip_month=12 and trip_day=31 AND taxi_type='yellow' GROUP BY trip_year,trip_month,trip_day ORDER BY trip_year,trip_month,trip_day;").show(truncate=False)

+---------+----------+--------+---------------+
|trip_year|trip_month|trip_day|avg_tips       |
+---------+----------+--------+---------------+
|2021     |12        |31      |2.4135832640000|
+---------+----------+--------+---------------+



In [18]:
spark.sql(f"SELECT count(*) as trip_count FROM {DATABASE_NAME}.{MOR_TABLE_NAME} WHERE trip_year=2021 and trip_month=12 and trip_day=31 GROUP BY trip_year,trip_month,trip_day ORDER BY trip_year,trip_month,trip_day").show(truncate=False)

+----------+
|trip_count|
+----------+
|72199     |
+----------+



This concludes unit 4, please proceed to the next notebook.

In [19]:
%%javascript
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>